오늘의 과제  
1. 어제꺼 전처리(불용어 제거->내가 만든 불용어 사전으로)  
2. 모델 구조 변경 및 성능 개선 gogo 성능 개선 여부는 정확도(accuracy)로 판단!  
3. 우리나라 소설책 중 일부를 발췌하여 모델링 연습(선택) -> 토지 추천해줌(국립국어원) -> 형태소 분석 x 단순히 띄어쓰기로, 한자도 버려 -> 명사만 뽑아서 하든가


In [ ]:
#불용어 삭제해주는 함수 구현
def removeSW():
  return


이진 분류일 때는 sigmoid때문에 기본적인 cost함수인 MSE 안 씀 -> 대신 역함수인 로그함수로 cost함수 ㄱㄱ

In [1]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
texts = ['자연어 처리 알고리즘', '자연어 처리 방법', '자연어 NLP 알고리즘 알고리즘', '자연어 처리 전문가']

In [3]:
tok = Tokenizer() #클래스(설계도), 객체(건물)

In [4]:
tok.fit_on_texts(texts)

In [5]:
tok.index_word

{1: '자연어', 2: '처리', 3: '알고리즘', 4: '방법', 5: 'nlp', 6: '전문가'}

In [6]:
tok.texts_to_matrix(texts)

array([[0., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0.],
       [0., 1., 1., 0., 0., 0., 1.]])

In [7]:
tok.texts_to_matrix(texts, mode='binary')

array([[0., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0.],
       [0., 1., 1., 0., 0., 0., 1.]])

In [9]:
tok.texts_to_matrix(texts, mode='count')

array([[0., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 0., 1., 0., 0.],
       [0., 1., 0., 2., 0., 1., 0.],
       [0., 1., 1., 0., 0., 0., 1.]])

In [10]:
tok.texts_to_matrix(texts, mode='tfidf')

array([[0.        , 0.58778666, 0.69314718, 0.84729786, 0.        ,
        0.        , 0.        ],
       [0.        , 0.58778666, 0.69314718, 0.        , 1.09861229,
        0.        , 0.        ],
       [0.        , 0.58778666, 0.        , 1.43459998, 0.        ,
        1.09861229, 0.        ],
       [0.        , 0.58778666, 0.69314718, 0.        , 0.        ,
        0.        , 1.09861229]])

In [11]:
tok.texts_to_matrix(texts, mode='freq')#위의 3개가 잘 쓰이고 얘는 잘 안 쓰임

array([[0.        , 0.33333333, 0.33333333, 0.33333333, 0.        ,
        0.        , 0.        ],
       [0.        , 0.33333333, 0.33333333, 0.        , 0.33333333,
        0.        , 0.        ],
       [0.        , 0.25      , 0.        , 0.5       , 0.        ,
        0.25      , 0.        ],
       [0.        , 0.33333333, 0.33333333, 0.        , 0.        ,
        0.        , 0.33333333]])

In [8]:
import pandas as pd
import numpy as np
from string import punctuation

In [14]:
df = pd.read_csv("/content/NYT_2018.csv")

In [15]:
# headline 열의 값 추출
headline = list(df.headline.values)
headline

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?',
 'Commuter Reprogramming',
 'Unknown',
 'Unknown',
 'Ford Changed Leaders, Looking for a Lift. It’s Still Looking.',
 'Romney Failed to Win at Utah Convention, But Few Believe He’s Doomed',
 'Chain Reaction',
 'He Forced the Vatican to Investigate Sex Abuse. Now He’s Meeting With Pope Francis.',
 'In Berlin, artists find a home',
 'Unknown',
 'The Right Stuff',
 'Jimmy Carter Knows What North Korea Wants',
 'The Truth Is Out There',
 'New Jersey Ruling Could Reignite Battle Over Church-State Separation',
 'Procrastinating',
 'Word + Quiz: dilatory',
 'My Life-Threatening Bout With E. Coli Food Poisoning',
 'Choosing Brexit, a Town Yearned for Its Seafa

In [16]:
sum(df.headline=='Unknown')
len([w for w in headline if w=='Unknown'])      #=='Unknown'    #=='Unknown'

110

##  하려고 하는 것
- headline에 저장된 뉴스기사 제목을 다음 단어를 생성하는 LSTM기반 모델을 설계
- 동작방식 예
- 입력 :I, 생성하고자 하는 단어의 갯수
- 출력 :I was ...완성

"""
동작순서
x in headline -> pre_func(x) -> pre_func(title) -> for w in title
->if w not in punctuation -> w.lower() -> ''.join -> 소문자로 변환된 단어들이 연결됨
-> return res -> pre_headline리스트의 요소로 저장됨
"""

In [17]:
headline=[w for w in headline if w!='Unknown']
len(headline)
# 전처리
headline[0]
headline[300]

'Air Pollution May Pose Serious Risks to Young Children'

In [31]:
punctuation
# punctuation를 제거하기

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
def pre_func(title):
    #소문자 변환
    res=''.join(w.lower() for w in title if w not in punctuation)
    return res

In [18]:
pre_headline=[pre_func(x) for x in headline]

In [ ]:
pre_headline

In [19]:
tok=Tokenizer()
tok.fit_on_texts(pre_headline)

In [20]:
tok.index_word
len(tok.index_word)

3619

In [21]:
vocab_size=len(tok.index_word)+1

In [22]:
sequences=[]
for s in pre_headline:
  enc=tok.texts_to_sequences([s])[0]
  for i in range(1, len(enc)):
    seq=enc[:i+1]
    sequences.append(seq)

In [23]:
sequences

[[95, 263],
 [95, 263, 1100],
 [95, 263, 1100, 1101],
 [95, 263, 1100, 1101, 572],
 [95, 263, 1100, 1101, 572, 50],
 [95, 263, 1100, 1101, 572, 50, 7],
 [95, 263, 1100, 1101, 572, 50, 7, 2],
 [95, 263, 1100, 1101, 572, 50, 7, 2, 365],
 [95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10],
 [95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10, 1102],
 [96, 3],
 [96, 3, 1103],
 [96, 3, 1103, 2],
 [96, 3, 1103, 2, 14],
 [96, 3, 1103, 2, 14, 573],
 [96, 3, 1103, 2, 14, 573, 85],
 [96, 3, 1103, 2, 14, 573, 85, 1104],
 [96, 3, 1103, 2, 14, 573, 85, 1104, 366],
 [96, 3, 1103, 2, 14, 573, 85, 1104, 366, 367],
 [96, 3, 1103, 2, 14, 573, 85, 1104, 366, 367, 5],
 [96, 3, 1103, 2, 14, 573, 85, 1104, 366, 367, 5, 1105],
 [1, 14],
 [1, 14, 1106],
 [1, 14, 1106, 1107],
 [16, 2],
 [16, 2, 574],
 [16, 2, 574, 4],
 [16, 2, 574, 4, 210],
 [16, 2, 574, 4, 210, 368],
 [16, 2, 574, 4, 210, 368, 1108],
 [16, 2, 574, 4, 210, 368, 1108, 2],
 [16, 2, 574, 4, 210, 368, 1108, 2, 369],
 [16, 2, 574, 4, 210, 368, 1108, 2, 369, 57

In [44]:
sequences[:15]
# ex) lstm모델로 단어를 예측하도록 설계합니다.
입력                            출력
lstm                          모델로
lstm 모델로                     단어를
...
lstm 모델로 단어를 예측하도록      설계합니다.
# 처음 입력 -> 출력 -> 두 개를 합쳐서(lstm 모델로) 다시 인풋 -> 단어 하나 생성 -> 또 합치고 인풋 출력 반복

[[96, 264],
 [96, 264, 1101],
 [96, 264, 1101, 1102],
 [96, 264, 1101, 1102, 573],
 [96, 264, 1101, 1102, 573, 51],
 [96, 264, 1101, 1102, 573, 51, 7],
 [96, 264, 1101, 1102, 573, 51, 7, 2],
 [96, 264, 1101, 1102, 573, 51, 7, 2, 366],
 [96, 264, 1101, 1102, 573, 51, 7, 2, 366, 11],
 [96, 264, 1101, 1102, 573, 51, 7, 2, 366, 11, 1103],
 [97, 3],
 [97, 3, 1104],
 [97, 3, 1104, 2],
 [97, 3, 1104, 2, 15],
 [97, 3, 1104, 2, 15, 574]]

In [45]:
pre_headline[0]

'former nfl cheerleaders’ settlement offer 1 and a meeting with goodell'

In [24]:
sents = [['programmer', 'person'], ['programmer','word', 'secret']]

In [42]:
mytok = Tokenizer()
mytok.fit_on_texts(sents)
print(mytok.word_index)
print(mytok.texts_to_sequences(sents))

{'programmer': 1, 'person': 2, 'word': 3, 'secret': 4}
[[1, 2], [1, 3, 4]]


## lstm매개변수 return_sequences=False -> 맨 마지막에서만 출력 다대일 만약 번역기면 True로  


In [46]:
tok.word_index# 빈도수 순서대로 출력됨

{'the': 1,
 'a': 2,
 'to': 3,
 'of': 4,
 'in': 5,
 'for': 6,
 'and': 7,
 'unknown': 8,
 'on': 9,
 'is': 10,
 'with': 11,
 'trump': 12,
 'as': 13,
 'at': 14,
 'new': 15,
 'from': 16,
 'how': 17,
 'it': 18,
 'an': 19,
 'be': 20,
 'that': 21,
 'season': 22,
 'us': 23,
 'you': 24,
 'what': 25,
 'episode': 26,
 'can': 27,
 'your': 28,
 'he': 29,
 'not': 30,
 'his': 31,
 'are': 32,
 'teaching': 33,
 'out': 34,
 'was': 35,
 'no': 36,
 'by': 37,
 'trump’s': 38,
 'has': 39,
 'war': 40,
 'over': 41,
 'may': 42,
 'into': 43,
 'more': 44,
 'now': 45,
 'why': 46,
 'who': 47,
 'about': 48,
 'recap': 49,
 'activities': 50,
 '1': 51,
 'we': 52,
 'do': 53,
 'women': 54,
 'when': 55,
 'syria': 56,
 'trade': 57,
 'it’s': 58,
 '2': 59,
 'or': 60,
 'will': 61,
 'i': 62,
 'have': 63,
 'just': 64,
 'but': 65,
 'home': 66,
 'up': 67,
 'this': 68,
 'one': 69,
 'off': 70,
 'house': 71,
 'president': 72,
 'gop': 73,
 'our': 74,
 'long': 75,
 'facebook': 76,
 'case': 77,
 'life': 78,
 'end': 79,
 'some': 80,
 'po

In [25]:
type(tok.word_index)

dict

In [26]:
tok.word_index.items()

dict_items([('the', 1), ('a', 2), ('to', 3), ('of', 4), ('in', 5), ('for', 6), ('and', 7), ('on', 8), ('is', 9), ('with', 10), ('trump', 11), ('as', 12), ('at', 13), ('new', 14), ('from', 15), ('how', 16), ('it', 17), ('an', 18), ('be', 19), ('that', 20), ('season', 21), ('us', 22), ('you', 23), ('what', 24), ('episode', 25), ('can', 26), ('your', 27), ('he', 28), ('not', 29), ('his', 30), ('are', 31), ('teaching', 32), ('out', 33), ('was', 34), ('no', 35), ('by', 36), ('trump’s', 37), ('has', 38), ('war', 39), ('over', 40), ('may', 41), ('into', 42), ('more', 43), ('now', 44), ('why', 45), ('who', 46), ('about', 47), ('recap', 48), ('activities', 49), ('1', 50), ('we', 51), ('do', 52), ('women', 53), ('when', 54), ('syria', 55), ('trade', 56), ('it’s', 57), ('2', 58), ('or', 59), ('will', 60), ('i', 61), ('have', 62), ('just', 63), ('but', 64), ('home', 65), ('up', 66), ('this', 67), ('one', 68), ('off', 69), ('house', 70), ('president', 71), ('gop', 72), ('our', 73), ('long', 74), ('

In [27]:
idx2word = {}
for k, v in tok.word_index.items():
  idx2word[v] = k
print(idx2word)

{1: 'the', 2: 'a', 3: 'to', 4: 'of', 5: 'in', 6: 'for', 7: 'and', 8: 'on', 9: 'is', 10: 'with', 11: 'trump', 12: 'as', 13: 'at', 14: 'new', 15: 'from', 16: 'how', 17: 'it', 18: 'an', 19: 'be', 20: 'that', 21: 'season', 22: 'us', 23: 'you', 24: 'what', 25: 'episode', 26: 'can', 27: 'your', 28: 'he', 29: 'not', 30: 'his', 31: 'are', 32: 'teaching', 33: 'out', 34: 'was', 35: 'no', 36: 'by', 37: 'trump’s', 38: 'has', 39: 'war', 40: 'over', 41: 'may', 42: 'into', 43: 'more', 44: 'now', 45: 'why', 46: 'who', 47: 'about', 48: 'recap', 49: 'activities', 50: '1', 51: 'we', 52: 'do', 53: 'women', 54: 'when', 55: 'syria', 56: 'trade', 57: 'it’s', 58: '2', 59: 'or', 60: 'will', 61: 'i', 62: 'have', 63: 'just', 64: 'but', 65: 'home', 66: 'up', 67: 'this', 68: 'one', 69: 'off', 70: 'house', 71: 'president', 72: 'gop', 73: 'our', 74: 'long', 75: 'facebook', 76: 'case', 77: 'life', 78: 'end', 79: 'some', 80: 'power', 81: 'say', 82: 'white', 83: '‘the', 84: 'after', 85: 'its', 86: 'still', 87: 'right',

In [51]:
sequences

[[96, 264],
 [96, 264, 1101],
 [96, 264, 1101, 1102],
 [96, 264, 1101, 1102, 573],
 [96, 264, 1101, 1102, 573, 51],
 [96, 264, 1101, 1102, 573, 51, 7],
 [96, 264, 1101, 1102, 573, 51, 7, 2],
 [96, 264, 1101, 1102, 573, 51, 7, 2, 366],
 [96, 264, 1101, 1102, 573, 51, 7, 2, 366, 11],
 [96, 264, 1101, 1102, 573, 51, 7, 2, 366, 11, 1103],
 [97, 3],
 [97, 3, 1104],
 [97, 3, 1104, 2],
 [97, 3, 1104, 2, 15],
 [97, 3, 1104, 2, 15, 574],
 [97, 3, 1104, 2, 15, 574, 86],
 [97, 3, 1104, 2, 15, 574, 86, 1105],
 [97, 3, 1104, 2, 15, 574, 86, 1105, 367],
 [97, 3, 1104, 2, 15, 574, 86, 1105, 367, 368],
 [97, 3, 1104, 2, 15, 574, 86, 1105, 367, 368, 5],
 [97, 3, 1104, 2, 15, 574, 86, 1105, 367, 368, 5, 1106],
 [1, 15],
 [1, 15, 1107],
 [1, 15, 1107, 1108],
 [17, 2],
 [17, 2, 575],
 [17, 2, 575, 4],
 [17, 2, 575, 4, 211],
 [17, 2, 575, 4, 211, 369],
 [17, 2, 575, 4, 211, 369, 1109],
 [17, 2, 575, 4, 211, 369, 1109, 2],
 [17, 2, 575, 4, 211, 369, 1109, 2, 370],
 [17, 2, 575, 4, 211, 369, 1109, 2, 370, 57

In [28]:
ml=max(len(i) for i in sequences) # 24개의 단어로 이루어진 문장이 제일 길다.

In [35]:
sequences[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   95,
        263, 1100], dtype=int32)

In [29]:
sequences=pad_sequences(sequences,maxlen=ml, padding='pre') #maxlen의 기본값이 제일 긴 녀석으로 되도록 설정되어 있음, padding도 pre로 기본

In [63]:
sequences[0] #전체 문장들에 대해서 맨마지막만 y로 넣어

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,  96, 264], dtype=int32)

In [36]:
x=[]
y=[]
for i in sequences:
  x.append(i[:-1])
  y.append(i[-1])
x = sequences[:, :-1]
y = sequences[:,-1]

In [37]:
x.shape

(7809, 23)

In [83]:
y.shape

(7809,)

In [84]:
y

array([ 264, 1101, 1102, ...,  111,    2, 1015], dtype=int32)

In [39]:
y=to_categorical(y, vocab_size)

In [111]:
y.shape

(7809, 3621)

In [88]:
vocab_size

3621

In [32]:
#모델 구현
model = Sequential()
model.add(Embedding(vocab_size, 10)) #3621차원을 10차원으로 임베딩
model.add(LSTM(128))#여기서 128의 의미는 특성의 개수 정도로 이해해라 -> 연산 결과를 몇 가지의 특성으로 나타낼지
model.add(Dense(vocab_size, activation =  'softmax'))

embedding으로 차원을 줄이면 약간의 정보손실은 있으나, 연산이 많이 가벼워짐

In [100]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 10)          36210     
                                                                 
 lstm_3 (LSTM)               (None, 128)               71168     
                                                                 
 dense_2 (Dense)             (None, 3621)              467109    
                                                                 
Total params: 574,487
Trainable params: 574,487
Non-trainable params: 0
_________________________________________________________________


array([[   0,    0,    0, ...,    0,    0,   96],
       [   0,    0,    0, ...,    0,   96,  264],
       [   0,    0,    0, ...,   96,  264, 1101],
       ...,
       [   0,    0,    0, ...,  217,   10, 3620],
       [   0,    0,    0, ...,   10, 3620,  111],
       [   0,    0,    0, ..., 3620,  111,    2]], dtype=int32)

In [40]:
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=200, verbose=1)

Epoch 1/200
245/245 [==============================] - 19s 49ms/step - loss: 7.6766 - accuracy: 0.0293
Epoch 2/200
245/245 [==============================] - 3s 11ms/step - loss: 7.1513 - accuracy: 0.0303
Epoch 3/200
245/245 [==============================] - 2s 6ms/step - loss: 7.0101 - accuracy: 0.0334
Epoch 4/200
245/245 [==============================] - 2s 7ms/step - loss: 6.8714 - accuracy: 0.0393
Epoch 5/200
245/245 [==============================] - 2s 7ms/step - loss: 6.7018 - accuracy: 0.0456
Epoch 6/200
245/245 [==============================] - 2s 8ms/step - loss: 6.5171 - accuracy: 0.0480
Epoch 7/200
245/245 [==============================] - 2s 8ms/step - loss: 6.3214 - accuracy: 0.0561
Epoch 8/200
245/245 [==============================] - 1s 6ms/step - loss: 6.1287 - accuracy: 0.0595
Epoch 9/200
245/245 [==============================] - 1s 6ms/step - loss: 5.9416 - accuracy: 0.0631
Epoch 10/200
245/245 [==============================] - 1s 5ms/step - loss: 5.7667 - acc

In [ ]:
1. 입력 -> '나는' + 단어 10개 예상
2.
3. 나는 -> 모델 -> 나는 지금 -> 모델-> 나는 지금 집에 -> 모델 -> ... -> 모델 -> 나는 지금 집에 있는데 티비를 시청하고 있다.
4.

In [63]:
def gen_sent(model, tok, c_word, n):
  pred_sent=''
  for _ in range(n):
    enc = tok.texts_to_sequences([c_word])[0]
    #print(enc)
    enc=pad_sequences([enc], maxlen=ml, padding='pre')
    res = model.predict(enc)
    res= np.argmax(res)

    for w, i in tok.word_index.items():
      if i==res:
        print(w)# The end 예측됨
        c_word+= " " + w # The -> The end
        break
  print(c_word)
  #pred_sent+=c_word
  #return pred_sent

In [65]:
gen_sent(model, tok, 'Go', 10)#the는 시작단어, 10은 시작 단어 뒤로 몇 개 예측할지

1/1 [==============================] - 0s 22ms/step
to
1/1 [==============================] - 0s 20ms/step
face
1/1 [==============================] - 0s 20ms/step
to
1/1 [==============================] - 0s 19ms/step
a
1/1 [==============================] - 0s 20ms/step
pulitzer
1/1 [==============================] - 0s 20ms/step
renovation
1/1 [==============================] - 0s 19ms/step
with
1/1 [==============================] - 0s 19ms/step
gusto
1/1 [==============================] - 0s 20ms/step
the
1/1 [==============================] - 0s 20ms/step
70
Go to face to a pulitzer renovation with gusto the 70
